<table style="font-size: 1em; padding: 0; margin: 0;">

<tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #182AEB; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #ffffff;">Deep Learning </span> for Satellite Image Classification</span> (Manning Publications)<br/>by <em>Daniel Buscombe</em></strong><br/><br/>
        <strong>> Chapter 1: Deliverable Solution </strong><br/>
    </p>  

At first, we import the required libraries:

In [ ]:
%matplotlib inline
import tensorflow as tf
import requests
import rasterio
import matplotlib.pyplot as plt
from skimage import io

Then, we will download an image from the Sentinel2 dataset, similar to what we did in the Notebook that introduced this liveProject. As the image is quite big, this step can take a while to complete.

In [ ]:
URL = "https://lp-prod-resources.s3.us-west-2.amazonaws.com/106/example_TCI_10m.jp2"
response = requests.get(URL)
open("test_img.jp2", "wb").write(response.content)

And from it we will create three ```TIFF``` images, using the same function ```write_image``` as in the previous notebook: 

In [ ]:
def write_image(input, output, band):
    with rasterio.open(input, driver='JP2OpenJPEG') as src_dataset:
        
        array = src_dataset.read(band)
        # Get a copy of the source dataset's profile. Thus our
        # destination dataset will have the same dimensions,
        # number of bands, data type, and georeferencing as the
        # source dataset.
        kwds = src_dataset.profile

        # Change the format driver for the destination dataset to
        # 'GTiff', short for GeoTIFF.
        kwds['driver'] = 'GTiff'

        # Add GeoTIFF-specific keyword arguments.
        kwds['tiled'] = True
        kwds['blockxsize'] = 256
        kwds['blockysize'] = 256
        kwds['photometric'] = 'YCbCr'
        kwds['compress'] = 'JPEG'

        with rasterio.open(output, 'w', **kwds) as dst_dataset:
            # Write data to the destination dataset.
            dst_dataset.write(array.astype(rasterio.uint8), 1)

input_file = 'test_img.jp2'          
file_list = ['r1.tif', 'r2.tif', 'r3.tif']
[write_image(input_file, f, band) for f,band in zip(file_list, [1,2,3])]

We will now use only the first image we created. We read it using ```imread``` from ```scikit-image``` and display its dimensions

In [ ]:
img = io.imread('r1.tif')
print(img.shape)

The image is indeed huge! It has 3 color channels and is 10980 pixels in both veritcal and horizontal dimenstion. We will use ```tf.image``` to rescale it to a smaller image

In [ ]:
img_array = tf.image.resize(
    img,
    (640,640),
    preserve_aspect_ratio=True,
    antialias=False,
    name=None)

... then we will also rotate the image 90 degrees and store the rotated version in a new variable

In [ ]:
img_r = tf.image.rot90(img_array)

We are finally ready to display the result

In [ ]:
plt.subplot(121)
plt.imshow(img_array.numpy().astype('uint8'), cmap='gray')
plt.subplot(122)
plt.imshow(img_r.numpy().astype('uint8'), cmap='gray')